# Find solution

In [ ]:
from zubora_gabora.aco.aco_zubora_gabora import ACOZuboraGabora

n_blades = 10
times = {
    "Zu": {"F": 5, "G": 3},
    "Ga": {"F": 2, "G": 6}
}
aco = ACOZuboraGabora(n_blades, times, alpha=0.5, beta=1.0, rho=0.75)
aco.optimize(1000)
print(f"Best fitness: {1.0/aco.best_fitness:.0f}")
ACOZuboraGabora.decode_solution(aco.best_solution)

## Draw Gantt of solution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

def draw_gantt_tasks_ordered(assignments, schedule):
    """
    Draws a Gantt chart with each row representing a task in the order:
    F1, G1, F2, G2, …, Fn, Gn.
    
    The color of each bar indicates the worker (e.g., Zu, Ga) who performed the task.
    
    Parameters:
      - assignments: list of dictionaries for each sword mapping task ('F' or 'G') to a worker.
      - schedule: list of dictionaries for each sword mapping task ('F' or 'G') to [start, end] times.
    """
    # Define colors for each worker.
    worker_colors = {'Zu': 'tab:blue', 'Ga': 'tab:orange'}
    
    # Build a list of tasks in the desired order.
    tasks_list = []
    for sword_idx, (assign, times) in enumerate(zip(assignments, schedule)):
        # Process tasks in fixed order: F then G.
        for task in ['F', 'G']:
            worker = assign[task]
            start, end = times[task]
            duration = end - start
            label = f"{task}{sword_idx + 1}"
            tasks_list.append({
                'label': label,
                'worker': worker,
                'start': start,
                'duration': duration
            })
    
    # Create the plot.
    fig, ax = plt.subplots(figsize=(10, 6))
    row_height = 8
    row_gap = 10  # vertical spacing between rows
    
    # Plot each task.
    for row_idx, task_item in enumerate(tasks_list):
        y_pos = row_idx * row_gap
        ax.broken_barh(
            [(task_item['start'], task_item['duration'])],
            (y_pos, row_height),
            facecolors=worker_colors.get(task_item['worker'], 'gray'),
            edgecolor='black', alpha=0.8
        )
        # Add the task label inside the bar.
        ax.text(
            task_item['start'] + task_item['duration'] / 2, 
            y_pos + row_height / 2,
            task_item['label'], ha='center', va='center', color='white', fontsize=9
        )
    
    # Set y-ticks and labels.
    ax.set_yticks([i * row_gap + row_height / 2 for i in range(len(tasks_list))])
    ax.set_yticklabels([task['label'] for task in tasks_list])
    ax.set_xlabel("Time")
    ax.set_title("Gantt Chart (Ordered as F1, G1, F2, G2, …)")
    
    # Create a legend mapping colors to workers.
    legend_handles = [plt.Rectangle((0, 0), 1, 1, color=worker_colors[w]) for w in worker_colors]
    ax.legend(legend_handles, worker_colors.keys(), title="Worker", loc='upper right')
    
    plt.tight_layout()
    plt.show()


# Example data (as provided)
assignments = aco.best_solution[0]

_, schedule = aco._simulate_forging(assignments, aco.best_solution[1])

# Draw the ordered Gantt chart.
draw_gantt_tasks_ordered(ACOZuboraGabora._decode_who(assignments), schedule)

# Process Visualization

## Fitness evolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

fitness = 1/np.array([[ant[1] for ant in trails] for trails in aco.trails_history ])
best_fitness = 1/np.array(aco.best_fitness_history)

fig, axs = plt.subplots(figsize=(5,5))
axs.set_title('Fitness evolution')
axs.set_xlabel('Iterations')
axs.set_ylabel('Fitness')

axs.plot(best_fitness, label='best_high')

median = np.median(fitness, axis=1)
min_array = np.min(fitness, axis=1)
max_array = np.max(fitness, axis=1)
axs.plot(median, label='iterations_high')
axs.fill_between(np.arange(len(median)), min_array, max_array, alpha=0.3, color='orange')

plt.legend()

## Diversity evolution

In [ ]:
population = np.array([[np.array(ant[0]+ant[1]) for ant,_ in trails] for trails in aco.trails_history ])

diversity = np.sum(np.std(population, axis=1), axis=1)
fig, axs = plt.subplots(figsize=(5,5))
axs.set_title('Diversity evolution')
axs.set_xlabel('Iterations')
axs.set_ylabel('Diversity')
axs.plot(diversity, color='orange')

## Number of different solutions

In [ ]:
from itertools import accumulate

population = np.array([[np.array(ant[0]+ant[1]) for ant,_ in trails] for trails in aco.trails_history ])

def stringify_individual(individual) -> str:
    return ''.join([str(int(i)) for i in individual])

a = np.apply_along_axis(stringify_individual, 2, population)
a = list(accumulate(a, lambda x, y: x.union(set(y)), initial=set()))

fig, axs = plt.subplots(figsize=(5,5))
axs.plot([len(x) for x in a], color='orange')
axs.set_title('Unique solutions evolution')
axs.set_xlabel('Iterations')
axs.set_ylabel('Unique solutions')

## Pheromone evolution

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# -----------------------------
# Example: Create sample data.
# Replace this with your own list of arrays.
list_of_arrays = aco.pheromone_history
# -----------------------------

# Create a figure and axis for plotting.
fig, ax = plt.subplots(1,2, figsize=(12, 5))


def plot_heatmap(data, ax):
    ax.clear()  # Clear previous heatmap

    # Normalize the array: (data - min) / (max - min)
    data_min = data.min()
    data_max = data.max()
    # Prevent division by zero in case data is constant
    if data_max - data_min != 0:
        norm_data = (data - data_min) / (data_max - data_min)
    else:
        norm_data = np.zeros_like(data)
    
    # Plot the normalized heatmap. Remove color bar with cbar=False.
    sns.heatmap(norm_data, ax=ax, cmap="inferno", cbar=False, vmin=0, vmax=1)

def update(frame):
    """Update function for the animation.
    
    This function normalizes the current array's values between 0 and 1,
    clears the previous plot, and plots the normalized heatmap.
    """
    
    plot_heatmap(list_of_arrays[frame][0], ax[0])
    plot_heatmap(list_of_arrays[frame][1], ax[1])
    
    fig.suptitle(f"Frame {frame+1}")
    

# Create the animation. Adjust 'interval' (ms) and 'frames' as needed.
ani = animation.FuncAnimation(fig, update, frames=len(list_of_arrays), interval=50, repeat=False)

# Save the animation to a video file.
# Note: Ensure you have ffmpeg installed on your system.
writer = animation.FFMpegWriter(fps=5)
ani.save("heatmap_evolution_normalized.mp4", writer=writer)

# Optionally, display the animation window.
plt.show()